In [1]:
import torch
import pandas

from tqdm import tqdm

import src.datasets
import src.transforms
import src.models
import src.utils

In [2]:
DATA = "./data/"
MODELS = "./models/"
sdf = pandas.read_csv(DATA+"sample_submission.csv")

### Create Segmented Test Data

In [3]:
import importlib
import time
import hpacellseg.cellsegmentator as cellsegmentator
from src import utils

In [4]:
NUC_MODEL = "./segmentation/nuclei-model.pth"
CELL_MODEL = "./segmentation/cell-model.pth"

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    padding=True,
    multi_channel_model=True,
)

No GPU found, using CPU.


please compile abn


/home/RDC/pfeuffem.hub/.conda/envs/hpa-cpu-env/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'pytorch_zoo.unet.DPNUnet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/RDC/pfeuffem.hub/.conda/envs/hpa-cpu-env/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/RDC/pfeuffem.hub/.conda/envs/hpa-cpu-env/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequenti

In [5]:
cells_frame = src.utils.create_cell_segmentation_rles(
    csv_file=DATA+"sample_submission.csv",
    root_dir=DATA+"/test/",
    segmentator=segmentator,
    test=True,
    n_samples=2,
    n_workers=24    
)
cells_frame.to_csv(DATA+f"test_cells_KAGGLE.csv", index=False)

Globbing ./data//test/.


2it [00:00, 48.09it/s]

Processing first 2 images.


Finished.


In [8]:
import torchvision

transforms = torchvision.transforms.Compose([
    src.transforms.MaskCrop(),
    src.transforms.PadToSquare(),
    src.transforms.Resize(224),
    src.transforms.ToTensor(),
    src.transforms.Normalize(),
])

ssdata = src.datasets.SingleCellDatasetKaggle(csv_file='data/test_cells_KAGGLE.csv', root_dir=DATA+"test/",
                                        transform=transforms, test=True)

### Create Predictions

In [9]:
ssloader = torch.utils.data.DataLoader(ssdata, batch_size=64, num_workers=8, collate_fn=src.datasets.collate_fn)

In [10]:
model = torch.load(MODELS+"model.pt")

In [11]:
def build_prediction_string(pred, conf, mask):
    mask = mask[2:-1]
    prediction_string = f"{pred} {conf} {mask}" 
    return prediction_string

In [12]:
with tqdm(ssloader, unit="batch") as tloader:
    tloader.set_description(f"Submission Sample")
    # Iterate over data.
    for images, labels, annotations in tloader:
        images = images.float().to(device)
        labels = labels.float().to(device)
        outputs = model_ft(images)
        confs, preds = torch.max(1/(1+torch.exp(-outputs)),1)
        for conf, pred, annotation in zip(confs, preds, annotations):
            iid = annotation['ID']
            pstring = build_prediction_string(pred, conf, annotation['rle'])
            sdf.loc[iid, 'PredictionString'] = sdf.loc[iid, 'PredictionString'] + ' ' + pstring

Submission Sample:   0%|          | 0/2 [00:00<?, ?batch/s]


NameError: Caught NameError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/RDC/pfeuffem.hub/.conda/envs/hpa-cpu-env/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/RDC/pfeuffem.hub/.conda/envs/hpa-cpu-env/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/RDC/pfeuffem.hub/.conda/envs/hpa-cpu-env/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/net/pfeuffem.hub/projects/hpa-single-cell-image-clasification/src/datasets.py", line 341, in __getitem__
    mt = io.imread(base_path + "_red.png")
NameError: name 'io' is not defined


In [ ]:
sdf.to_csv("./submissions/submission.csv")